Taken from: https://github.com/vspruyt/FastAhrs/blob/main/notebooks/AccelerometerCalibration.ipynb

Python 3 implementation of an accelerometer calibration procedure based on the paper 
"Autocalibration of MEMS Accelerometers" by Frosio et al.
This allows you to calibrate the accelerometer without having to perfectly align the 
accelerometer to a flat surface of any kind.
You just have to do a bunch of measurements with the accelerometer held in various random 
different orientations while holding it still.

The approach is based on the fact that the magnitude of each measurement (i.e. `sqrt(x^2 + y^2 + z^2)`) 
is supposed to be equal to 1 G when there is no linear acceleration going on. This, we can define
a non-linear error function, which can then be minimized by the Gauss-Newton algorithm.

The mathematical model used is `A = M(V - B)`, where `A` is the unknown calibrated acceleration,
while `V` is the measured acceleration which is skewed by unknown bias `B` and scale factor `M`.
An optimization procedure is used to find `M` and `B`, such that `sum((|A|-1)^2)` over all 
samples `A`, is minimized.

Note: `M` does not have to be a diagonal matrix. Therefore, this calibration procedure 
also accounts for cross-talk between the accelerometer axes by estimating the 
off-diagonal elements of `M`.


In [3]:
import pandas as pd
import numpy as np
import scipy.optimize as optimize # Tested with scipy 1.5.2
from io import StringIO

In [109]:
# Parameters to optimize
params = {
    'M': np.array([[1.0, 0.0, 0.0],
                   [0.0, 1.0, 0.0],
                   [0.0, 0.0, 1.0]]),
    'B': np.array([0.0, 0.0, 0.0])
}

def flatten_params(params):
    return np.hstack([params['M'].flatten(), params['B']])

def unflatten_params(param_vec):
    offset = 0
    boundary = params['M'].shape[0] * params['M'].shape[1]
    M = param_vec[offset:boundary].reshape(params['M'].shape[0], -1)

    offset = boundary
    boundary += params['B'].shape[0]
    B = param_vec[offset:boundary]
    return {'M': M, 'B': B}

param_vec = flatten_params(params)
param_vec

array([1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.])

In [110]:
unflatten_params(param_vec)

{'M': array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 'B': array([0., 0., 0.])}

In [140]:
# Input data is supposed to be in units of standard gravity
# The data file contains three columns. The first is the header: x, y, z.
# To capture data from an Arduino/Teensy that is connected via USB,
# just log to Serial and pipe the output to a file.
# If you are using PlatformIO, this is done by the the following
# terminal command: 'pio device monitor | tee data_sample.csv'.
# We remove lines that contain '>>>' symbols, because
# those are used in our C++ app to output information while
# logging data.
data_lines = ["x,y,z"]
with open('data_sample.csv', 'r') as infile:
    for line in infile:
        if (">>>" not in line
            and "---" not in line
            and len(line.strip())>0
            and line.strip()[0] in "-0123456789"):
            data_lines.append(line.strip())
(len(data_lines), data_lines[:4])

(12129,
 ['x,y,z',
  '0.04370,-0.05054,1.14746',
  '0.04614,-0.05005,1.15039',
  '0.03882,-0.04663,1.14453'])

In [141]:
data_df = pd.read_csv(StringIO('\n'.join(data_lines)))
data_df

,x,y,z
0,0.04370,-0.05054,1.14746
1,0.04614,-0.05005,1.15039
2,0.03882,-0.04663,1.14453
3,0.04956,-0.05078,1.13403
4,0.04443,-0.04639,1.13916
...,...,...,...
12123,0.05518,-0.03833,1.14038
12124,0.05444,-0.03979,1.14111
12125,0.05396,-0.04004,1.13892
12126,0.05615,-0.03906,1.14673


In [132]:
params['M'].dot((data_df.values - params['B']).transpose()).transpose()

array([[ 0.04883, -0.052  ,  1.14917],
       [ 0.04346, -0.05176,  1.14478],
       [ 0.04028, -0.04736,  1.14282],
       ...,
       [ 0.04541, -0.04834,  1.14966],
       [ 0.03955, -0.04468,  1.14795],
       [ 0.04639, -0.0481 ,  1.14478]])

In [133]:
np.matmul(params['M'], (data_df.values - params['B']).transpose()).transpose()

array([[ 0.04883, -0.052  ,  1.14917],
       [ 0.04346, -0.05176,  1.14478],
       [ 0.04028, -0.04736,  1.14282],
       ...,
       [ 0.04541, -0.04834,  1.14966],
       [ 0.03955, -0.04468,  1.14795],
       [ 0.04639, -0.0481 ,  1.14478]])

In [157]:
# Define the loss function to be minimized
def get_unbiased_values(param_vec, sensor_vals_matrix):
    params = unflatten_params(param_vec)
    return params['M'].dot((data_df.values - params['B']).transpose()).transpose()


def loss_function(param_vec, sensor_vals):
    unbiased_values = get_unbiased_values(param_vec, sensor_vals)
    squared_norm = np.sum(unbiased_values**2,axis=-1)
    tmp_params = unflatten_params(param_vec)
    M = tmp_params['M']
    # The last part of this error is a small regularization term which takes into account the
    # norm of the off-diagonal elements of the scaling matrix.
    # This ensures that we try to find a solution that doesn't skew the raw measurements too much,
    # and helps a lot when it comes to avoiding overfitting.
    # Try to set the weight (0.1) to zero, and see what happens to the result.
    err = abs(squared_norm - 1) + \
          0.08*np.linalg.norm([M[0, 1], M[0, 2], M[1, 0], M[1, 2], M[2, 0], M[2, 1]])
    return err


# Minimize the sum, over all samples, of the square of the loss function
result = optimize.least_squares(loss_function, param_vec,
                                args=(data_df.values,),
                                loss='soft_l1', f_scale=0.1,
                                verbose=2, ftol=1e-11,
                               )
if result.success:
    print('Success!')
else:
    print('Solution not found!')

print(f"Number of evaluations: {result.nfev}")
print(result.message)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.2721e+02                                    1.36e+03    
       1              4         1.1438e+01      1.16e+02       1.08e-01       2.67e+02    
       2              6         2.7370e+00      8.70e+00       2.71e-02       1.21e+02    
       3              7         4.1188e-01      2.33e+00       5.41e-02       5.68e+01    
       4              9         3.0422e-01      1.08e-01       2.71e-02       6.90e+00    
       5             10         2.3388e-01      7.03e-02       2.71e-02       4.63e+00    
       6             12         2.1023e-01      2.36e-02       6.77e-03       1.78e+00    
       7             15         2.0860e-01      1.63e-03       8.46e-04       7.08e-01    
       8             17         2.0834e-01      2.59e-04       4.23e-04       2.71e-01    
       9             19         2.0834e-01      2.68e-06       2.11e-04       3.16e-01    

In [158]:
calibration_params = unflatten_params(result.x)

print('Calibration done!')
print('To calibrate your new samples, use [new_x, new_y, new_z] = ([raw_x, raw_y, raw_z]-B)M')
print('')
print('M:')
print(calibration_params['M'])
print('')
print('B:')
print(calibration_params['B'])


Calibration done!
To calibrate your new samples, use [new_x, new_y, new_z] = ([raw_x, raw_y, raw_z]-B)M

M:
[[9.97643554e-01 9.49082059e-05 8.79251628e-04]
 [9.48468209e-05 9.97001949e-01 2.25321705e-04]
 [8.71089265e-04 2.23376965e-04 9.88382490e-01]]

B:
[ 0.02550119 -0.00721161  0.13346816]


In [146]:
# Use the following printed lines for our Arduino/Teensy C-code normalization
# by pasting it in CalibrationEEPROM_read_write.cpp.

print("let acc_misalignment = FusionMatrix::new(", end="")
els = calibration_params['M'].flatten()
for ix, el in enumerate(els):
    print(f"{el:.6}", end="")
    if ix < len(els)-1:
        print(", ", end="")
        if ix in [2,5]:
            print("\n                                         ", end="")

print(");")

print("let acc_offset = FusionVector::new(", end="")
print(f"{calibration_params['B'][0]:.6}f32, {calibration_params['B'][1]:.6}f32, {calibration_params['B'][2]:.6}f32", end="");
print(");")

let acc_misalignment = FusionMatrix::new(0.997644f32, 9.51672e-05f32, 0.000879306f32, 
                                         9.5087e-05f32, 0.997002f32, 0.000225248f32, 
                                         0.000871131f32, 0.000223482f32, 0.988382f32);
let acc_offset = FusionVector::new(0.0255012f32, -0.00721161f32, 0.133468f32);
